# Prepare Data

### Import

In [8]:
from glob import glob

import pandas as pd

# data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

#Modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

### Explore

In [33]:
def wrangle(filepath):
    '''
    This function 
    
    input: Filepath for dataset to used
    output: Returns a clean dataframe for analysis
    '''
    
    df = pd.read_csv(filepath)
    #subsetting for Nigeria only
    mask_NIG = df['Area'] == 'Nigeria'
    df = df[mask_NIG]
    
    
    #creating a pivot table
#     df = pd.pivot_table(df, index = "Year" , columns = "Item", values ="Value")
    
    return df

In [6]:
files = glob('*.csv') 
files

['crop1.csv', 'live1.csv', 'pop1.csv']

In [37]:
to_be_used = ['crop1.csv', 'live1.csv']
frame = [wrangle(file) for file in to_be_used]
frame = pd.concat(frame, ignore_index= True)
print(frame.shape)
frame.head()

(7795, 6)


,Area,Item,Element,Year,Unit,Value
0,Nigeria,"Bastfibres, other",Area harvested,1981,ha,1000.0
1,Nigeria,"Bastfibres, other",Area harvested,1982,ha,1000.0
2,Nigeria,"Bastfibres, other",Area harvested,1983,ha,1000.0
3,Nigeria,"Bastfibres, other",Area harvested,1984,ha,1000.0
4,Nigeria,"Bastfibres, other",Area harvested,1985,ha,1000.0


6

In [32]:
frame = wrangle('live1.csv')
print(frame.shape)
frame

(60, 10)


Item,Asses,Beehives,Camels,Cattle,Chickens,Goats,Horses,Pigs,Rabbits and hares,Sheep
Year,,,,,,,,,,
1961,1398000.0,NaN,14000.0,6028000.0,37360.0,623000.0,281500.0,640000.0,NaN,1019000.0
1962,1364000.0,NaN,14500.0,6311000.0,38710.0,758000.0,272000.0,660000.0,NaN,1165000.0
1963,1330000.0,NaN,14500.0,6627000.0,40150.0,917000.0,262500.0,680000.0,NaN,1341000.0
1964,1296000.0,NaN,15000.0,6946000.0,41650.0,1113000.0,253000.0,700000.0,NaN,1544000.0
1965,1240000.0,NaN,16000.0,7236000.0,43240.0,1336000.0,351000.0,720000.0,NaN,1751000.0
1966,984654.0,NaN,16500.0,7516000.0,44890.0,1595000.0,356333.0,740000.0,NaN,1982000.0
1967,900000.0,NaN,17000.0,7828000.0,46640.0,1903000.0,350000.0,770000.0,NaN,2241000.0
1968,880000.0,NaN,18000.0,8190000.0,48470.0,2267000.0,345000.0,800000.0,NaN,2513000.0
1969,860000.0,NaN,18000.0,8578000.0,50410.0,2732000.0,340000.0,826000.0,NaN,2868000.0


In [22]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 542 entries, 63942 to 64483
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Area     542 non-null    object 
 1   Item     542 non-null    object 
 2   Element  542 non-null    object 
 3   Year     542 non-null    int64  
 4   Unit     542 non-null    object 
 5   Value    498 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 29.6+ KB


### Split

# Build Model

### Baseline

### Iterate

### Evaluate

# Communicate Results